# TP2 - Clustering (Avec Réduction de Dimension)

Ce notebook charge les données Hi-Seq normalisées depuis `data/`. Il applique d'abord une réduction de dimension (PCA et UMAP) à 100 composantes, puis applique K-Means, DBSCAN et Spectral Clustering sur ces données réduites.

In [ ]:
# --- Imports ---
from pathlib import Path
import sys
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering
from sklearn.neighbors import NearestNeighbors
from kneed import KneeLocator
import warnings

In [3]:
# --- Configuration des Chemins ---

PROJECT_ROOT = Path("..").resolve()
DATA_PROCESSED_PATH = PROJECT_ROOT / "data" / "HISEQ"
PROJECT_FCT = PROJECT_ROOT / "src"
sys.path.append(str(PROJECT_FCT))

print(f"Chemin du projet racine : {PROJECT_ROOT}")
print(f"Chargement des données depuis : {DATA_PROCESSED_PATH}")

Chemin du projet racine : C:\Users\sebdr\OneDrive\Bureau\sherbrooke\IFT 599 - Sciences des données\TP2 Devoir\git
Chargement des données depuis : C:\Users\sebdr\OneDrive\Bureau\sherbrooke\IFT 599 - Sciences des données\TP2 Devoir\git\data\HISEQ


In [4]:
# --- Chargement des modules locaux ---

try:
    from preprocess import reduce_dimension_for_clustering
    from utils import (
        evaluate_clustering, 
        plot_clusters,
        run_stochastic_protocol, 
        print_protocol_summary, 
        run_deterministic_protocol
    )
    print("Modules locaux 'utils' et 'preprocess' chargés avec succès.")
except ImportError:
    print("Attention: Les fichiers utils.py ou preprocess.py n'ont pas été trouvés dans /src.")
    print("Définition de fonctions de remplacement (placeholders).")
    
warnings.filterwarnings('ignore')

c:\Users\sebdr\anaconda3\envs\udes_sd_a25\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Modules locaux 'utils' et 'preprocess' chargés avec succès.


In [5]:
# --- Chargement des données ---
print("Chargement des données Hi-Seq normalisées...")
try:
    X_scaled = np.load(DATA_PROCESSED_PATH / 'hiseq_X_scaled.npy')
    y_hiseq = pd.read_csv(DATA_PROCESSED_PATH / 'hiseq_y_labels.csv', index_col=0).iloc[:, 0]
    print(f"Données chargées. Forme de X_scaled: {X_scaled.shape}")
except FileNotFoundError:
    print(f"ERREUR: Fichiers non trouvés dans {DATA_PROCESSED_PATH}")
    print("Veuillez d'abord exécuter le notebook 'PreparationData.ipynb'.")

Chargement des données Hi-Seq normalisées...
Données chargées. Forme de X_scaled: (801, 20531)


# 1. Réduction de Dimension (100 composantes)

 Pour ce protocole, nous effectuons la RD une seule fois avec une graine fixe (42).
 *Note : Un protocole encore plus rigoureux pourrait inclure la réduction UMAP (qui est stochastique) à l'intérieur de la boucle de 10 exécutions.*

In [6]:
N_COMPONENTS = 100

# --- PCA (100) ---
# X_scaled est un np.array, ce qui est compatible avec votre fonction
X_pca_100 = reduce_dimension_for_clustering(X_scaled, method='PCA', n_components=N_COMPONENTS)

# --- UMAP (100) ---
X_umap_100 = reduce_dimension_for_clustering(X_scaled, method='UMAP', n_components=N_COMPONENTS)

Début de la réduction de dimension avec PCA à 100 composantes...
Réduction PCA terminée. Nouvelle forme : (801, 100)
Début de la réduction de dimension avec UMAP à 100 composantes...
Réduction UMAP terminée. Nouvelle forme : (801, 100)


In [9]:
# --- Constantes pour le protocole ---
N_RUNS = 10
N_CLUSTERS = 5 # 5 classes dans HiSeq

# 2. Clustering sur Données Réduites (PCA)

============ 2.1 K-Means (PCA) - Protocole expérimental (10 exécutions) ============

In [10]:
# Définir la "fabrique" (factory) pour KMeans
kmeans_factory = lambda seed: KMeans(n_clusters=N_CLUSTERS, random_state=seed, n_init=10)

# Exécuter le protocole
df_kmeans_pca = run_stochastic_protocol(
    model_factory=kmeans_factory,
    data=X_pca_100,
    true_labels=y_hiseq,
    n_runs=N_RUNS,
    experiment_name="KMeans (PCA)",
    evaluate_clustering_func=evaluate_clustering
)

--- Protocole expérimental : KMeans (PCA) ---
  Exécution 1/10 (KMeans (PCA))...
  Exécution 2/10 (KMeans (PCA))...
  Exécution 3/10 (KMeans (PCA))...
  Exécution 4/10 (KMeans (PCA))...
  Exécution 5/10 (KMeans (PCA))...
  Exécution 6/10 (KMeans (PCA))...
  Exécution 7/10 (KMeans (PCA))...
  Exécution 8/10 (KMeans (PCA))...
  Exécution 9/10 (KMeans (PCA))...
  Exécution 10/10 (KMeans (PCA))...
Protocole KMeans (PCA) terminé.


In [ ]:
# Afficher le résumé
results_kmeans_pca_mean = print_protocol_summary(df_kmeans_pca, "KMeans (PCA)")


--- Résultats KMeans (PCA) - (μ ± σ) sur 10 exécutions ---
Silhouette:        0.2049 ± 0.0004
Homogeneity:       0.8619 ± 0.0032
Completeness:      0.8479 ± 0.0031
V-Measure:         0.8548 ± 0.0032
ARI:               0.7963 ± 0.0067
Time (s):          0.2630 ± 0.4787
Memory (MB):       1.3872 ± 0.0953


============ 2.2 DBSCAN (PCA) ============


DBSCAN est déterministe. Une seule exécution suffit.

In [12]:
print("Calcul de EPS pour DBSCAN (PCA)...")
neighbors_pca = NearestNeighbors(n_neighbors=5).fit(X_pca_100)
distances_pca, _ = neighbors_pca.kneighbors(X_pca_100)
distances_pca = np.sort(distances_pca, axis=0)[:, 4]
kneedle_pca = KneeLocator(range(len(distances_pca)), distances_pca, curve='convex', direction='increasing')
EPS_PCA_OPTIMAL = kneedle_pca.elbow_y if kneedle_pca.elbow_y else 126.0
print(f"EPS Optimal (PCA) trouvé: {EPS_PCA_OPTIMAL}")

Calcul de EPS pour DBSCAN (PCA)...
EPS Optimal (PCA) trouvé: 126.53904060021102


In [22]:
dbscan_pca_model = DBSCAN(eps=EPS_PCA_OPTIMAL, min_samples=5)

# Exécuter le protocole déterministe
results_dbscan_pca_series = run_deterministic_protocol(
    model=dbscan_pca_model,
    data=X_pca_100,
    true_labels=y_hiseq,
    experiment_name="DBSCAN (PCA)",
    evaluate_clustering_func=evaluate_clustering
)

--- Exécution : DBSCAN (PCA) ---
Exécution DBSCAN (PCA) terminée.

--- Résultats DBSCAN (PCA) - (1 exécution) ---
|                  |        0 |
|:-----------------|---------:|
| Silhouette       | nan      |
| Homogeneity      |   0.0000 |
| Completeness     |   1.0000 |
| V-Measure        |   0.0000 |
| ARI              |   0.0000 |
| Noise_Percentage |   5.3683 |
| Time (s)         |   0.0192 |
| Memory (MB)      |   2.0861 |


============ 2.3 Clustering Spectral (PCA) - Protocole expérimental (10 exécutions) ============

In [15]:
# Définir la "fabrique" pour Spectral Clustering
spectral_factory = lambda seed: SpectralClustering(n_clusters=N_CLUSTERS, assign_labels='kmeans', random_state=seed)

In [16]:
# Exécuter le protocole
df_spectral_pca = run_stochastic_protocol(
    model_factory=spectral_factory,
    data=X_pca_100,
    true_labels=y_hiseq,
    n_runs=N_RUNS,
    experiment_name="Spectral (PCA)",
    evaluate_clustering_func=evaluate_clustering
)

--- Protocole expérimental : Spectral (PCA) ---
  Exécution 1/10 (Spectral (PCA))...
  Exécution 2/10 (Spectral (PCA))...
  Exécution 3/10 (Spectral (PCA))...
  Exécution 4/10 (Spectral (PCA))...
  Exécution 5/10 (Spectral (PCA))...
  Exécution 6/10 (Spectral (PCA))...
  Exécution 7/10 (Spectral (PCA))...
  Exécution 8/10 (Spectral (PCA))...
  Exécution 9/10 (Spectral (PCA))...
  Exécution 10/10 (Spectral (PCA))...
Protocole Spectral (PCA) terminé.


In [17]:
# Afficher le résumé
results_spectral_pca_mean = print_protocol_summary(df_spectral_pca, "Spectral (PCA)")


--- Résultats Spectral (PCA) - (μ ± σ) sur 10 exécutions ---
Silhouette:        -0.0430 ± 0.0154
Homogeneity:       0.0070 ± 0.0020
Completeness:      0.0195 ± 0.0065
V-Measure:         0.0101 ± 0.0025
ARI:               0.0008 ± 0.0075
Time (s):          0.4756 ± 0.4106
Memory (MB):       20.9940 ± 0.1360


# 3. Clustering sur Données Réduites (UMAP)

============ 3.1 K-Means (UMAP) - Protocole expérimental (10 exécutions) ============

In [18]:
# La "fabrique" kmeans_factory est la même que précédemment
df_kmeans_umap = run_stochastic_protocol(
    model_factory=kmeans_factory, # On réutilise la même fabrique
    data=X_umap_100,              # Mais sur des données différentes
    true_labels=y_hiseq,
    n_runs=N_RUNS,
    experiment_name="KMeans (UMAP)",
    evaluate_clustering_func=evaluate_clustering
)

--- Protocole expérimental : KMeans (UMAP) ---
  Exécution 1/10 (KMeans (UMAP))...
  Exécution 2/10 (KMeans (UMAP))...
  Exécution 3/10 (KMeans (UMAP))...
  Exécution 4/10 (KMeans (UMAP))...
  Exécution 5/10 (KMeans (UMAP))...
  Exécution 6/10 (KMeans (UMAP))...
  Exécution 7/10 (KMeans (UMAP))...
  Exécution 8/10 (KMeans (UMAP))...
  Exécution 9/10 (KMeans (UMAP))...
  Exécution 10/10 (KMeans (UMAP))...
Protocole KMeans (UMAP) terminé.


In [19]:
results_kmeans_umap_mean = print_protocol_summary(df_kmeans_umap, "KMeans (UMAP)")


--- Résultats KMeans (UMAP) - (μ ± σ) sur 10 exécutions ---
Silhouette:        0.8964 ± 0.0000
Homogeneity:       0.9901 ± 0.0000
Completeness:      0.9913 ± 0.0000
V-Measure:         0.9907 ± 0.0000
ARI:               0.9925 ± 0.0000
Time (s):          0.1193 ± 0.0139
Memory (MB):       1.1659 ± 0.0461


============ 3.2 DBSCAN (UMAP) ============

In [20]:
print("Calcul de EPS pour DBSCAN (UMAP)...")
neighbors_umap = NearestNeighbors(n_neighbors=5).fit(X_umap_100)
distances_umap, _ = neighbors_umap.kneighbors(X_umap_100)
distances_umap = np.sort(distances_umap, axis=0)[:, 4]
kneedle_umap = KneeLocator(range(len(distances_umap)), distances_umap, curve='convex', direction='increasing')
EPS_UMAP_OPTIMAL = kneedle_umap.elbow_y if kneedle_umap.elbow_y else 0.42
print(f"EPS Optimal (UMAP) trouvé: {EPS_UMAP_OPTIMAL}")

Calcul de EPS pour DBSCAN (UMAP)...
EPS Optimal (UMAP) trouvé: 0.4881645143032074


In [21]:
# Initialiser le modèle
dbscan_umap_model = DBSCAN(eps=EPS_UMAP_OPTIMAL, min_samples=5)

# Exécuter le protocole déterministe
results_dbscan_umap_series = run_deterministic_protocol(
    model=dbscan_umap_model,
    data=X_umap_100,
    true_labels=y_hiseq,
    experiment_name="DBSCAN (UMAP)",
    evaluate_clustering_func=evaluate_clustering
)

--- Exécution : DBSCAN (UMAP) ---
Exécution DBSCAN (UMAP) terminée.

--- Résultats DBSCAN (UMAP) - (1 exécution) ---
|                  |      0 |
|:-----------------|-------:|
| Silhouette       | 0.8965 |
| Homogeneity      | 0.9945 |
| Completeness     | 0.9951 |
| V-Measure        | 0.9948 |
| ARI              | 0.9963 |
| Noise_Percentage | 0.2497 |
| Time (s)         | 0.0211 |
| Memory (MB)      | 0.9517 |


============ 3.3 Clustering Spectral (UMAP) - Protocole expérimental (10 exécutions) ============

In [23]:
# La "fabrique" spectral_factory est la même
df_spectral_umap = run_stochastic_protocol(
    model_factory=spectral_factory, # On réutilise la même fabrique
    data=X_umap_100,               # Mais sur des données différentes
    true_labels=y_hiseq,
    n_runs=N_RUNS,
    experiment_name="Spectral (UMAP)",
    evaluate_clustering_func=evaluate_clustering
)

--- Protocole expérimental : Spectral (UMAP) ---
  Exécution 1/10 (Spectral (UMAP))...
  Exécution 2/10 (Spectral (UMAP))...
  Exécution 3/10 (Spectral (UMAP))...
  Exécution 4/10 (Spectral (UMAP))...
  Exécution 5/10 (Spectral (UMAP))...
  Exécution 6/10 (Spectral (UMAP))...
  Exécution 7/10 (Spectral (UMAP))...
  Exécution 8/10 (Spectral (UMAP))...
  Exécution 9/10 (Spectral (UMAP))...
  Exécution 10/10 (Spectral (UMAP))...
Protocole Spectral (UMAP) terminé.


In [24]:
# Afficher le résumé
results_spectral_umap_mean = print_protocol_summary(df_spectral_umap, "Spectral (UMAP)")


--- Résultats Spectral (UMAP) - (μ ± σ) sur 10 exécutions ---
Silhouette:        0.3848 ± 0.2247
Homogeneity:       0.7673 ± 0.1401
Completeness:      0.8738 ± 0.0649
V-Measure:         0.8146 ± 0.1101
ARI:               0.6465 ± 0.1970
Time (s):          0.2788 ± 0.1096
Memory (MB):       21.5483 ± 0.0447


# 4. Comparaison Finale

In [25]:
# Combiner le tout
data_summary = {
    ('PCA', 'KMeans'): results_kmeans_pca_mean,
    ('PCA', 'DBSCAN'): results_dbscan_pca_series,
    ('PCA', 'Spectral'): results_spectral_pca_mean,
    ('UMAP', 'KMeans'): results_kmeans_umap_mean,
    ('UMAP', 'DBSCAN'): results_dbscan_umap_series,
    ('UMAP', 'Spectral'): results_spectral_umap_mean
}

In [26]:
df_final_results = pd.DataFrame(data_summary).T
df_final_results.index.names = ['Jeu', 'Algo']

print("\n--- Tableau Récapitulatif (Moyennes des 10 exécutions) ---")
columns_order = ['ARI', 'V-Measure', 'Silhouette', 'Noise_Percentage', 'Time (s)', 'Memory (MB)']
# S'assurer que toutes les colonnes existent avant de les réorganiser
final_columns = [col for col in columns_order if col in df_final_results.columns]
print(df_final_results[final_columns].to_markdown(floatfmt=".4f"))


--- Tableau Récapitulatif (Moyennes des 10 exécutions) ---
|                      |    ARI |   V-Measure |   Silhouette |   Noise_Percentage |   Time (s) |   Memory (MB) |
|:---------------------|-------:|------------:|-------------:|-------------------:|-----------:|--------------:|
| ('PCA', 'KMeans')    | 0.7963 |      0.8548 |       0.2049 |           nan      |     0.2630 |        1.3872 |
| ('PCA', 'DBSCAN')    | 0.0000 |      0.0000 |     nan      |             5.3683 |     0.0192 |        2.0861 |
| ('PCA', 'Spectral')  | 0.0008 |      0.0101 |      -0.0430 |           nan      |     0.4756 |       20.9940 |
| ('UMAP', 'KMeans')   | 0.9925 |      0.9907 |       0.8964 |           nan      |     0.1193 |        1.1659 |
| ('UMAP', 'DBSCAN')   | 0.9963 |      0.9948 |       0.8965 |             0.2497 |     0.0211 |        0.9517 |
| ('UMAP', 'Spectral') | 0.6465 |      0.8146 |       0.3848 |           nan      |     0.2788 |       21.5483 |
